# Surrogate Construction for Genz Functions: Regression

This notebook constructs a PC surrogate for Genz functions using full and sparse quadrature and gives the normalized root mean square error between the surrogate and the actual function. The Genz functions are defined on [-1,1].

In [1]:
import numpy as np
import math  
from scipy.stats import qmc
import pandas as pd

import PyUQTk.pce as uqtkpce
import PyUQTk.PyPCE.pce_tools as pce_tools
from PyUQTk.utils.func import *

PyMC is required for some of the MCMC postprocessing codes.
Will proceed without, but some convergence tests will not be available.


## Inputs

In [2]:
nord = 8            # Order of the PCE
pc_type = "LU"      # Polynomial type
pc_alpha = 0.0      # Free parameter > -1 for Gamma-Laguerre and Beta-Jacobi PCs
pc_beta = 1.0       # Free parameter > -1 for Gamma-Laguerre and Beta-Jacobi PCs
param = nord+1      # Number of quadrature points per dimension for full quadrature or level for sparse quadrature
ndim = 2            # Number of dimensions
model= 'genz_osc'   # Choices are 'genz_osc', 'genz_exp', 'genz_cont','genz_gaus','genz_cpeak', 'genz_ppeak'

We first define a PC object. for regression and then get a collection of random sample points in [-1,1].

In [3]:
pc_model = uqtkpce.PCSet("NISPnoq", nord, ndim, pc_type, pc_alpha, pc_beta)

Then, we generate training and testing data.

In [4]:
# Random generator
rng = qmc.LatinHypercube(d=ndim, seed=42)

# Training
nTrain=int(2*pc_model.GetNumberPCTerms())
x_train = 2*rng.random(n=nTrain)-1 #draw n samples from [-1,1]
y_train = func(x_train,model,np.ones(ndim+1))

# Testing
nTest=10000
x_test=2*rng.random(n=nTest)-1
y_test=func(x_test,model,np.ones(ndim+1))

Then, we use regression to find the PC coefficients and evaluate the PC at the testing points. The error between these evaluations and the actual model is determined.

In [5]:
#Regression
c_k = pce_tools.UQTkRegression(pc_model, y_train, x_train)

# Validation points
pce_evals = pce_tools.UQTkEvaluatePCE(pc_model, c_k, x_test)

#Error
MSE = np.square(np.subtract(y_test,pce_evals)).mean()
RMSE=math.sqrt(MSE)

print("The NRMS error between a", ndim, "-dimensional", model, "function and a regression-based PC surrogate of \
order", nord, "is")
RMSE/np.linalg.norm(y_test)

The NRMS error between a 2 -dimensional genz_osc function and a regression-based PC surrogate of order 8 is


8.441105764360879e-11